In [1]:
import pandas as pd
import datetime
from datetime import date, time, timedelta
import itertools
import time
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Global variables (CAPS)

battery_power = 300
battery_cap = 580
charge_eff = 90
discharge_eff = 90
mlf = 0.991

In [4]:
def raw_power(charge_forecast, discharge_forecast, opening_cap):
    if charge_forecast == 1 and discharge_forecast == 0:
        return -min(battery_power, (battery_cap - opening_cap)/(charge_eff/100)*2)
    elif charge_forecast == 0 and discharge_forecast == 1:
        return min(battery_power, (opening_cap/(discharge_eff/100))*2 )
    else:
        return 0

In [5]:
def market_dispatch(raw_power):
    if raw_power < 0:
        return (raw_power/2)
    elif raw_power > 0:
        return (raw_power/2)*discharge_eff/100
    else:
        return 0

In [6]:
def market_revenue(market_dispatch,spot_price):
    if market_dispatch < 0:
        return market_dispatch*spot_price*(1/mlf)
    elif market_dispatch > 0:
        return market_dispatch*spot_price*mlf
    else:
        return 0

In [7]:
def closing_capacity(market_dispatch, opening_cap):
    if market_dispatch < 0:
        x = opening_cap - market_dispatch * (charge_eff/100)
        return (max(0, min(round(x), battery_cap)))
    elif market_dispatch > 0:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))
    else:
        x = opening_cap - market_dispatch * (100/discharge_eff)
        return (max(0, min(round(x), battery_cap)))

In [35]:
def quantile_prices(index):
    if index <= (len(subsample)-10):  
        q1 = subsample.iloc[index+1:index + 11]['Spot Price'].quantile(0.20)
        q3 = subsample.iloc[index+1:index + 11]['Spot Price'].quantile(0.80)
    elif index > (len(subsample)-10):  #not ok
        q1 = subsample.iloc[index+1:len(subsample)]['Spot Price'].quantile(0.20)
        q3 = subsample.iloc[index+1:len(subsample)]['Spot Price'].quantile(0.80)
        
    return q1,q3

In [29]:
def charging_condition(index, current_price, q1, q3):
    ''' Determine whether current period is forcasted to charge or discharge '''  
    
    # Discharge conditions
    if (current_price < q3):
        subsample["Discharge Forecast"].values[index] = 0
    
    elif (current_price >= q3):
        subsample["Discharge Forecast"].values[index] = 1
        
    if (current_price > q1):
        subsample["Charge Forecast"].values[index] = 0
        
    elif (current_price <= q1):
        subsample["Charge Forecast"].values[index] = 1
    

In [10]:
df = pd.read_excel("../../data/market_data.xlsx") 

In [11]:
vic_spotprice = df.filter(items=['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)'])
vic_spotprice = vic_spotprice.rename(columns={'Regions VIC Trading Price ($/MWh)': 'Spot Price', 'Time (UTC+10)': 'Time'})
# Change column type to datetime type
vic_spotprice['Time'] = pd.to_datetime(vic_spotprice['Time'])

In [36]:
# Subsample first 3 days
subsample = vic_spotprice.loc[vic_spotprice['Time'].dt.date <= date(2020, 12, 31)]
# subsample

In [13]:
subsample

,Time,Spot Price
0,2018-01-01 00:00:00,90.43
1,2018-01-01 00:30:00,92.46
2,2018-01-01 01:00:00,87.62
3,2018-01-01 01:30:00,73.08
4,2018-01-01 02:00:00,70.18
...,...,...
63452,2021-08-14 22:00:00,49.93
63453,2021-08-14 22:30:00,62.86
63454,2021-08-14 23:00:00,32.26
63455,2021-08-14 23:30:00,25.10


In [37]:
# Create new columns

subsample["q1"] = 0.0
subsample["q3"] = 0.0
subsample["Charge Forecast"] = 0
subsample["Discharge Forecast"] = 0
subsample["Raw Power"] = 0
subsample["Market Dispatch"] = 0
subsample["Market Revenue"] = 0
subsample["Opening Capacity"] = 0
subsample["Closing Capacity"] = 0

In [38]:
%%time

for index, row in subsample.iterrows():
    q1,q3 = quantile_prices(index)
    subsample["q1"].values[index] = q1
    subsample["q3"].values[index] = q3
    
    current_price = subsample.iloc[index]["Spot Price"]

    charging_condition(index, current_price, q1, q3)
    
    if index != 0:
        subsample["Opening Capacity"].values[index] = subsample["Closing Capacity"].values[index-1]
    
    subsample["Raw Power"].values[index] = raw_power(subsample["Charge Forecast"].values[index], subsample["Discharge Forecast"].values[index], subsample["Opening Capacity"].values[index])
    subsample["Market Dispatch"].values[index] = market_dispatch(subsample["Raw Power"].values[index])
    subsample["Market Revenue"].values[index] = market_revenue(subsample["Market Dispatch"].values[index], subsample["Spot Price"].values[index])
    subsample["Closing Capacity"].values[index] = closing_capacity(subsample["Market Dispatch"].values[index], subsample["Opening Capacity"].values[index])

CPU times: user 1min 18s, sys: 17.3 ms, total: 1min 18s
Wall time: 1min 18s


In [39]:
# Current > Past  --> Discharge
# Current < Future --> Charge

# subsample.tail(50)
sum(subsample["Market Revenue"])

99235064

In [40]:
subsample.tail(50)

,Time,Spot Price,q1,q3,Charge Forecast,Discharge Forecast,Raw Power,Market Dispatch,Market Revenue,Opening Capacity,Closing Capacity
52558,2020-12-30 23:00:00,25.67,14.396,33.792,0,0,0,0,0,280,280
52559,2020-12-30 23:30:00,36.52,13.148,32.486,0,1,300,135,4885,280,130
52560,2020-12-31 00:00:00,36.66,13.148,23.658,0,1,288,129,4686,130,0
52561,2020-12-31 00:30:00,33.11,11.788,21.058,0,1,0,0,0,0,0
52562,2020-12-31 01:00:00,32.33,11.788,20.950,0,1,0,0,0,0,0
52563,2020-12-31 01:30:00,21.49,10.566,16.854,0,1,0,0,0,0,0
52564,2020-12-31 02:00:00,20.95,10.566,15.222,0,1,0,0,0,0,0
52565,2020-12-31 02:30:00,20.95,9.856,14.726,0,1,0,0,0,0,0
52566,2020-12-31 03:00:00,14.64,5.924,13.750,0,1,0,0,0,0,0
52567,2020-12-31 03:30:00,10.03,-5.380,13.750,0,0,0,0,0,0,0
